In [1]:
import tweepy
import json 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from datetime import datetime
import os 
import time 

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from config import (consumer_key,
                   consumer_secret,
                   access_token, 
                   access_token_secret)

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())


In [2]:
def tweet_request(tweet, tweet_data=dict()):
    tweet_data = []
    tweet_id = tweet['id']
    tweet_user = tweet['user']['screen_name']
    tweet_requests = []
    for mentions in tweet['entities']['user_mentions']:
        if(mentions['screen_name'] != "PySentimentBot"):
            tweet_requests.append(mentions["screen_name"])
            
    tweet_dict = {"id":tweet_id, "user":tweet_user,"analysis_requests":tweet_requests}
    return tweet_dict
    

In [3]:
def analyze_sentiments(recent_tweets, sentiment_results= list()):
    sentiment_results = []
    for tweet in recent_tweets: 
        new_tweet = cleanse_tweet(tweet)
        sentiment_tweet = analyzer.polarity_scores(new_tweet["text"])
        sentiment_tweet.update({"tweet_id":new_tweet["id"]})
        sentiment_tweet.append(sentiment_result)
    return sentiment_results

In [4]:
def color_map(value):
    if(value >= 0): 
        return 'g'
    else:
        return 'r'

def plot_sentiments(title,sentiments):
    df = pd.DataFrame(sentiments)
    df = df.reset_index()
    df.plot( 'index', 'compound', linestyle='-', marker='o',alpha=0.75)
    plt.ylabel("Sentiment score")
    plt.xlabel("Tweets")
    plt.title(title)
    
    filename = "SentimentAnalysis_of_"+title+".png"
    plt.savefig(filename)
    plt.show()
    
    return filename 

In [5]:
def scan_requests(since_tweet_id):
    total_tweets = 0 
    last_tweet = since_tweet_id
    search_handle = ''
    results = api.mentions_timeline(since_tweet_id)
    
    if(len(results) > 0):
        tweet_data = []
        
        for tweet in results:
            parsed_tweet = pare_requests(tweet)
            last_tweet_id = tweet['id']
            tweet_data.append(parsed_tweet)
            
        for item in tweet_data:
            
            recent_tweets = []
            for analyze_requests in item["analysis_requests"]:
                recent_tweets = api.user_timeline(analyze_requests, count = 200)
                print(f"{analyze_request} - {len(recent_tweets)}")
                
                if(len(recent_tweets) > 0):
                    sentiments = analyze_sentiments(recent_tweets)
                    sentiment_fig = plot_sentiments(analyze_requests, sentiments)
                    text_status = f"{datetime.now()} - Thank you for your tweet @{item['user']}! Here is the sentiment analysis of {analyze_request}!"
                    api.update_with_media(filename=sentiment_fig,status=text_status,in_reply_to_status_id=item["id"])
                else: 
                    text_status = f"{datetime.now()} Thank you for your tweet @{item['user']}! Sorry, {analyze_request} has no tweets!" 
                    api.update_status(text_status)
                    
                    total_tweets_so_far = total_tweets_so_far + 1
                plt.show()
            return last_tweet_id
        else:
            return last_tweet_id
                    

In [ ]:
since_tweet_id = 1020094176736579584

while True:
    time.sleep(30)
    since_tweet_id = scan_requests(since_tweet_id) 

In [ ]:
WSJ - 200

In [ ]:
CocaCola - 200